Assumption: The earnings date is taken as the ex-date

1. Get list of company tickers from finviz
2. Get earnings data report for each company in `companies`
3. Make a DF row for each company row

Columns to populate:
For each company: For each earnings
1. compute gap%
2. find C1
3. find O
4. find market cap
5. find distance from 52W high
6. find recency of 52W high
7. trading volume near announcement (at exdate)
8. trading volume before announcement (at c1)

FORMULAE:
1. Market cap = Total shares outstanding * price at that date
2. 52W_HIGH = 
3. 52W_RECENCY = 
4. 


CHANGES:
1. Change 52week defintion to 252 days instead of 364 days.
2. for the first week, do min (52 weeks, available weeks)


In [2]:
# import finviz
from finvizfinance.screener.custom import Custom

def get_us_stocks():
    fcustom = Custom()
    cols = [0, 1]
    filters = { 'Country': 'USA' }
    fcustom.set_filter(filters_dict=filters)
    df1 = fcustom.screener_view(columns=cols, select_page=1, sleep_sec=1, order='Ticker')
    return df1

def write_to_file(df, filename):
    df.to_csv(rf"{filename}", header=None, sep=' ', mode='w')

write_to_file(get_us_stocks(), "stocks/US_STOCKS.txt")
print("[+] Completed Task")

[+] Completed Taske [##############################] 1/1 


In [17]:
import time
import requests
import pandas as pd

BASE_URL = 'https://alphavantage.co'
AV_API_KEY = 'IHLNUQ0G66C0MJID'

def read_symbols(filename):
    with open(filename, "r") as file:
      data = file.read().split('\n')
      data = [d.split(' ')[1] for d in data]
    return data

def av_get_adjusted_ohlc(api_key, symbol):
    """ return adjusted OHLC for symbol for the year as a dataframe"""
    url = f"{BASE_URL}/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&datatype=json&outputsize=full&apikey={api_key}"

    response = requests.get(url)

    return pd.DataFrame(response.json()['Time Series (Daily)'])

def get_bulk_us_ohlc_data(filename, start_date='', end_date=''):
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
      df = av_get_adjusted_ohlc(api_key=AV_API_KEY, symbol=symbol)
      df = df.T      
      df['date'] = df.index
      if start_date and end_date:
        filtered_df = df.loc[(df['date'] > start_date) & (df['date'] < end_date)]
      else:
        filtered_df = df
      # write to a CSV file
      filtered_df.to_csv(rf"ohlc/{symbol}_ADJUSTED_OHLC.txt", header=filtered_df.columns, sep=',', mode='w')
      time.sleep(12) # limit is 5 requests / minute
      print(f"[+] {symbol} DONE")

def get_us_ohlc_data(symbol, start_date='', end_date=''):
    df = av_get_adjusted_ohlc(api_key=AV_API_KEY, symbol=symbol)
    df = df.T      
    df['date'] = df.index
    if start_date and end_date:
      filtered_df = df.loc[(df['date'] > start_date) & (df['date'] < end_date)]
    else:
      filtered_df = df
    # write to a CSV file
    filtered_df.to_csv(rf"ohlc/{symbol}_ADJUSTED_OHLC.txt", header=filtered_df.columns, sep=',', mode='w')
    print(f"[+] {symbol} DONE")

def av_get_earnings(api_key, ticker):
    ticker = ticker.strip().upper()

    url = f"{BASE_URL}/query?function=EARNINGS&symbol={ticker}&apikey={api_key}"
    response = requests.get(url)

    return pd.DataFrame(response.json()['quarterlyEarnings'])

get_bulk_us_ohlc_data("tmp/US_STOCKS.txt")
# get adjusted OHLC data for all US_stocks for the year 2022


KeyError: 'Time Series (Daily)'

In [139]:
import datetime

def get_bulk_us_earnings_date(filename):
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
        earnings_date = av_get_earnings(api_key=AV_API_KEY, ticker=symbol)
        earnings_date.to_csv(
            f"earnings/{symbol}_EARNINGS_DATE.txt", sep=',', mode='w', header=earnings_date.columns)
        time.sleep(12)


def av_get_company_overview(api_key, symbol):
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)

    return response.json()


def get_bulk_us_overview(filename):
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
        overview = av_get_company_overview(api_key=AV_API_KEY, symbol=symbol)
        with open(f"overview/{symbol}_OVERVIEW.json", "w") as f:
            f.write(overview)
        time.sleep(12)


def build_gap_table(filename):
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
        ohlc_df = pd.read_csv(f"data/{symbol}_ADJUSTED_OHLC.txt")
        earnings_dates = pd.read_csv(f"earnings/{symbol}_EARNINGS_DATE.txt")
        ohlc_df = ohlc_df.set_index('date')
        openings_at_ex_date_df = ohlc_df.loc[ohlc_df.index.isin(
            earnings_dates)]
        overview_json = pd.read_json(f"overview/{symbol}_OVERVIEW.txt")

        myrows = []
        # for each earnings date
        for index, row in openings_at_ex_date_df.iterrows():
            # find the closing date data before exdate
            if ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0]:
                c1 = ohlc_df.iloc[ohlc_df.index.get_loc(
                    ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0])+1]
            mydict = {}
            mydict['exdate'] = index
            mydict['Opening at ex date'] = row['1. open']
            mydict['C1 date'] = c1['Unnamed: 0']
            mydict['C1'] = c1['4. close']
            mydict['Gap%'] = (row['1. open'] / c1['4. close'] - 1) * 100
            mydict['Market Cap'] = overview_json['MarketCapitalization']
            mydict['52W_dist'] = c1['4. close'] - float(overview_json['52WeekHigh'])

            fiftytwowh_date_series = ohlc_df.loc[ohlc_df['2. high'] == float(
                overview_json['52WeekHigh'])]['Unnamed: 0']

            # TODO: Refactor
            if fiftytwowh_date_series is not None:
                c1_date = datetime.datetime.strptime(
                    c1['Unnamed: 0'], "%Y-%m-%d").date()
                fiftytwoWH_date = datetime.datetime.strptime(
                    fiftytwowh_date_series[0], "%Y-%m-%d").date()

                # doesn't make sense to calculate recency if NOT from the same year
                if c1_date.year == fiftytwoWH_date.year:
                    recency = c1_date - fiftytwoWH_date
                else:  
                    recency = ''
            else:
                recency = ''
            
            mydict['52W_recency'] = recency
            mydict['Trading vol at exdate'] = row['6. volume']

            myrows.append(mydict)
        gap = pd.DataFrame(myrows)
        gap.to_csv(f"{symbol}_GAP_TABLE.txt", header=gap.columns, sep=',', mode='w')
        print(f"[+] Constructed gap table for ${symbol}")


In [5]:
symbol = "META"
earnings_dates = av_get_earnings(AV_API_KEY, symbol)['reportedDate']
type(earnings_dates)

pandas.core.series.Series

In [6]:
ohlc_df = pd.read_csv(f"data/{symbol}_ADJUSTED_OHLC.txt")

In [7]:
ohlc_df = ohlc_df.set_index('date')
openings_at_ex_date_df = ohlc_df.loc[ohlc_df.index.isin(earnings_dates)]

In [8]:
def av_get_company_overview(api_key, symbol):
    symbol = symbol.strip().upper()
    
    url = f"{BASE_URL}/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)

    return response.json()

In [18]:
overview_json = av_get_company_overview(AV_API_KEY, "META")

ERROR! Session/line number was not unique in database. History logging moved to new session 15


In [87]:
symbol = "META"
def get_52_week_data(date):
    """
    Return the 52 week (52*7 = 364 days) high 
    Note: This function ignores the first 51 weeks of the historical OHLC as 52Wh is not defined
    """
    myrows = []
    for i in range(52*7):
      try:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Unnamed: 0'] == date].tolist()[0]) + i]
        myrows.append(c1)
      except IndexError:
         pass
    frame = pd.DataFrame(myrows)
    frame.to_csv("tmp/DATA.csv", mode='w', header=frame.columns)
    high = max(frame['2. high'])
    high_date = frame['2. high'].idxmax()
    low_date = frame['3. low'].idxmin()
    low = min(frame['3. low'])
    print(frame.shape)
    wh52 = frame.index.get_indexer_for([high_date])
    wl52 = frame.index.get_indexer_for([low_date])
    return (high, int(wh52), low, int(wl52))

print(get_52_week_data('2023-01-31'))

(364, 9)
(384.33, 355, 88.09, 58)


In [3]:
import datetime
myrows = []
for index, row in openings_at_ex_date_df.iterrows():
    if ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0]:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(
            ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0])+1]
    mydict = {}
    mydict['exdate'] = index
    mydict['O'] = row['1. open']
    mydict['C1 date'] = c1['Unnamed: 0']
    mydict['C1'] = c1['4. close']
    mydict['Gap%'] = (row['1. open'] / c1['4. close'] - 1) * 100


    # TODO: Market cap
    # 1. Market cap = Total shares outstanding * price at that date
    mydict['MktCap'] = float(overview_json['SharesOutstanding']) * float(row['1. open'])

    # TODO: Compute 52WH from OHLC data (52*7)
    # Formula: (52WH - Close) / 52WH
    WH_52, WH52_recency, WL_52, WL52_recency = get_52_week_data(c1['Unnamed: 0'])
    # print(WH_52, WH52_recency, WL_52, WL52_recency, "C1", c1['Unnamed: 0'])

    mydict['52WH_dist'] = (WH_52 - c1['4. close']) / WH_52
    mydict['52WL_dist'] = (c1['4. close'] - WH_52) / WL_52

    # TODO: 52WH_recency
    # Redo with respect to new defintion
    # fiftytwowh_date_series = ohlc_df.loc[ohlc_df['2. high'] == float(
    #     overview_json['52WeekHigh'])]['Unnamed: 0']
    # if fiftytwowh_date_series is not None:
    #     c1_date = datetime.datetime.strptime(
    #         c1['Unnamed: 0'], "%Y-%m-%d").date()
    #     fiftytwoWH_date = datetime.datetime.strptime(
    #         fiftytwowh_date_series[0], "%Y-%m-%d").date()
    #     if c1_date.year == fiftytwoWH_date.year:
    #         recency = c1_date - fiftytwoWH_date
    #     else:
    #         recency = ''
    # else:
    #     recency = ''

    # Formula: C1 date - 52WH date
    # mydict['52WH_recency'] = (datetime.datetime.strptime(c1['Unnamed: 0'], "%Y-%m-%d").date() - datetime.datetime.strptime(WH_52_date, "%Y-%m-%d").date()).days // 7
    # mydict['52WL_recency'] = (datetime.datetime.strptime(c1['Unnamed: 0'], "%Y-%m-%d").date() - datetime.datetime.strptime(WL_52_date, "%Y-%m-%d").date()).days// 7
    mydict['52WH_recency'] = WH52_recency
    mydict['52WL_recency'] = WL52_recency

    mydict['v0'] = row['6. volume']
    mydict['v1'] = c1['6. volume']
    # TODO: Get vol1 as well

    myrows.append(mydict)

gap = pd.DataFrame(myrows)


NameError: name 'openings_at_ex_date_df' is not defined

In [90]:
gap.to_csv("tmp/META_GAP_TABLE.csv", mode='w', sep=',', header=gap.columns)

In [15]:
gap.to_csv(f"META_GAP_TABLE.csv", mode="w", sep=",", header=gap.columns)